In [27]:
import keras
import numpy as np
import pandas as pd
import seaborn as sbn
import matplotlib
%matplotlib inline

In [28]:
pd.set_option('display.max_columns', None)
df=pd.read_csv('data/Existing_Commercial_Buildings_Energy_Performance_Ordinance_Report.csv', na_values='?')
df.head()

,Parcel(s),Building Name,Building Address,Postal Code,Full.Address,Floor Area,Property Type - Self Selected,PIM Link,Benchmark 2016 Status,2016 Reason for Exemption,Benchmark 2015 Status,2015 Reason for Exemption,Benchmark 2014 Status,2014 Reason for Exemption,Benchmark 2013 Status,2013 Reason for Exemption,Benchmark 2012 Status,2012 Reason for Exemption,Benchmark 2011 Status,2011 Reason for Exemption,Benchmark 2010 Status,2010 Reason for Exemption,Energy Audit Due Date,Energy Audit Status,Next Audit Due Date,Next Energy Audit Status,2016 ENERGY STAR Score,2016 Site EUI (kBtu/ft2),2016 Source EUI (kBtu/ft2),2016 Percent Better than National Median Site EUI,2016 Percentage Better than National Median Source EUI,2016 Total GHG Emissions (Metric Tons CO2e),2016 Total GHG Emissions Intensity (kgCO2e/ft2),2016 Weather Normalized Site EUI (kBtu/ft2),2016 Weather Normalized Source EUI (kBtu/ft2),2015 ENERGY STAR Score,2015 Site EUI (kBtu/ft2),2015 Source EUI (kBtu/ft2),2015 Percent Better than National Median Site EUI,2015 Percentage Better than National Median Source EUI,2015 Total GHG Emissions (Metric Tons CO2e),2015 Total GHG Emissions Intensity (kgCO2e/ft2),2015 Weather Normalized Site EUI (kBtu/ft2),2015 Weather Normalized Source EUI (kBtu/ft2),2014 ENERGY STAR Score,2014 Site EUI (kBtu/ft2),2014 Source EUI (kBtu/ft2),2014 Percent Better than National Median Site EUI,2014 Percent Better than National Median Source EUI,2014 Total GHG Emissions (Metric Tons CO2e),2014 Total GHG Emissions Intensity (kgCO2e/ft2),2014 Weather Normalized Site EUI (kBtu/ft2),2014 Weather Normalized Source EUI (kBtu/ft2),2013 ENERGY STAR Score,2013 Site EUI (kBtu/ft2),2013 Source EUI (kBtu/ft2),2013 Percent Better than National Median Site EUI,2013 Percent Better than National Median Source EUI,2013 Total GHG Emissions (Metric Tons CO2e),2013 Total GHG Emissions Intensity (kgCO2e/ft2),2013 Weather Normalized Site EUI (kBtu/ft2),2013 Weather Normalized Source EUI (kBtu/ft2),2012 ENERGY STAR Score,2012 Site EUI (kBtu/ft2),2012 Source EUI (kBtu/ft2),2012 Percent Better than National Median Site EUI,2012 Percent Better than National Median Source EUI,2012 Total GHG Emissions (Metric Tons CO2e),2012 Total GHG Emissions Intensity (kgCO2e/ft2),2012 Weather Normalized Site EUI (kBtu/ft2),2012 Weather Normalized Source EUI (kBtu/sq.ft),2011 ENERGY STAR Score,2011 Site EUI (kBtu/ft2),2011 Source EUI (kBtu/ft2),2011 Percent Better than National Median Site EUI,2011 Percent Better than National Median Source EUI,2011 Total GHG Emissions (Metric Tons CO2e),2011 Total GHG Emissions Intensity (kgCO2e/ft2),2011 Weather Normalized Site EUI (kBtu/ft2),2011 Weather Normalized Source EUI (kBtu/ft2)
0,7521/002,Diamond Heights Shopping Center - Building B2,"5262, 5264, 5268 Diamond Heights Blvd.",94131.0,"5262 5264\nSan Francisco, CA 94131\n",10728.0,Retail Store,http://propertymap.sfplanning.org/?&search=752...,Complied,NaN,Complied,NaN,Complied,NaN,Complied,NaN,Complied,NaN,Violation - Did Not Report,NaN,Violation - Did Not Report,NaN,2013-04-01T00:00:00,Complied,NaN,NaN,NaN,48.5,148.8,20.9,20.9,44.6,4.2,49.0,149.3,NaN,48.9,149.6,21.5,21.5,42.2,3.9,49.1,149.8,NaN,50.7,155.0,25.9,25.9,43.8,4.1,51.4,155.7,NaN,50.8,153.4,24.6,24.6,43.6,4.1,50.1,152.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0667/011,"1430. 1450, 1480 Van Ness Ave",1430 Van Ness,94109.0,"1430 Van\nSan Francisco, CA 94109\n",28300.0,Retail Store,http://propertymap.sfplanning.org/?&search=066...,Violation - Did Not Report,NaN,Complied,NaN,Complied,NaN,Complied,NaN,Complied,NaN,Complied,NaN,Exempt,Exempt - SqFt Not Subject This Year,2014-04-01T00:00:00,Complied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.9,87.3,-23.7,-23.7,37.9,2.3,27.9,87.3,NaN,28.9,90.7,-20.8,-20.8,39.4,2.4,28.9,90.7,NaN,32.7,99.0,-13.5,-13.5,43.8,2.6,32.7,99.0,NaN,25.5,75.3,-34.3,-34.2,33.70,2.0,25.5,75.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0742/015,700-710 Van Ness,700 Van Ness,94102.0,"700 Van\nSan 

In [29]:
all_cols=(df.columns)#Grab a list of all columns in the dataset
ident_cols=['Parcel(s)', 'Building Name', 'Building Address', 'Full.Address','PIM Link']#Not important.
cols_2016=['2016 ENERGY STAR Score', '2016 Site EUI (kBtu/ft2)',\
           '2016 Source EUI (kBtu/ft2)', '2016 Percent Better than National Median Site EUI',\
           '2016 Percentage Better than National Median Source EUI', \
           '2016 Total GHG Emissions (Metric Tons CO2e)', \
           '2016 Total GHG Emissions Intensity (kgCO2e/ft2)', \
           '2016 Weather Normalized Site EUI (kBtu/ft2)', '2016 Weather Normalized Source EUI (kBtu/ft2)']


cont_cols=[x for x in all_cols if df[x].dtype!="object" if x!="Postal Code"]#Get all continuous variables
categ_cols=[x for x in all_cols if df[x].dtype=="object" if x not in ident_cols]
categ_cols.append("Postal Code")#Not a continuous variable.

print ("       Identifier Columns")
print (ident_cols)
print ("")
print ("       Categorical Columns")
print (categ_cols)
print ("")
print ("       Continuous Columns")
print (cont_cols)

       Identifier Columns
['Parcel(s)', 'Building Name', 'Building Address', 'Full.Address', 'PIM Link']

       Categorical Columns
['Property Type - Self Selected', 'Benchmark 2016 Status', '2016 Reason for Exemption', 'Benchmark 2015 Status', '2015 Reason for Exemption', 'Benchmark 2014 Status', '2014 Reason for Exemption', 'Benchmark 2013 Status', '2013 Reason for Exemption', 'Benchmark 2012 Status', '2012 Reason for Exemption', 'Benchmark 2011 Status', '2011 Reason for Exemption', 'Benchmark 2010 Status', '2010 Reason for Exemption', 'Energy Audit Due Date', 'Energy Audit Status', 'Next Audit Due Date', 'Next Energy Audit Status', 'Postal Code']

       Continuous Columns
['Floor Area', '2016 ENERGY STAR Score', '2016 Site EUI (kBtu/ft2)', '2016 Source EUI (kBtu/ft2)', '2016 Percent Better than National Median Site EUI', '2016 Percentage Better than National Median Source EUI', '2016 Total GHG Emissions (Metric Tons CO2e)', '2016 Total GHG Emissions Intensity (kgCO2e/ft2)', '2016 

In [30]:
print ("Some missing data for these sets.")

#For the categorical values
fillUNK=False
if fillUNK:#Fill all categorical missing values with "UNK"
    df[categ_cols]=df[categ_cols].fillna("UNK")
else:#Fill all values with the mediod.
    df[categ_cols]=df[categ_cols].apply(lambda col : col.fillna(col.value_counts().index[0]))

#Fill in each column with the median value if missing for the continuous values
df[cont_cols]=df[cont_cols].apply(lambda col : col.fillna(col.median()))    

Some missing data for these sets.


In [31]:
print ("For each categorical column, build dummy variables.")
for i in categ_cols:
    print ("Building dummy variables for ",i)
    df=pd.get_dummies(df, prefix=i)

For each categorical column, build dummy variables.
Building dummy variables for  Property Type - Self Selected
Building dummy variables for  Benchmark 2016 Status
Building dummy variables for  2016 Reason for Exemption
Building dummy variables for  Benchmark 2015 Status
Building dummy variables for  2015 Reason for Exemption
Building dummy variables for  Benchmark 2014 Status
Building dummy variables for  2014 Reason for Exemption
Building dummy variables for  Benchmark 2013 Status
Building dummy variables for  2013 Reason for Exemption
Building dummy variables for  Benchmark 2012 Status
Building dummy variables for  2012 Reason for Exemption
Building dummy variables for  Benchmark 2011 Status
Building dummy variables for  2011 Reason for Exemption
Building dummy variables for  Benchmark 2010 Status
Building dummy variables for  2010 Reason for Exemption
Building dummy variables for  Energy Audit Due Date
Building dummy variables for  Energy Audit Status
Building dummy variables for  

In [32]:
target='2016 Total GHG Emissions (Metric Tons CO2e)'
print ("We are trying to predict the ",target)
X,y=df.drop(target,axis=1),df[[target]]
X=X.drop(ident_cols,axis=1)#Drop all of the identifying columns as well as the target variable
X=X.drop(cols_2016,axis=1)#We don't want any 2016 info influencing our decision.


We are trying to predict the  2016 Total GHG Emissions (Metric Tons CO2e)


In [33]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

RF=RandomForestRegressor(n_estimators=30, criterion='mse',n_jobs=-1)
MSEs=cross_val_score(RF, X, y[target], cv=10)

print ("Random Forest Regressor MSE on 10-fold CV= %0.2f +/- %0.2f" %(np.mean(MSEs),np.std(MSEs)))

Random Forest Regressor MSE on 10-fold CV= 0.80 +/- 0.22


This is almost too easy to predict! 

What if we restrict the information available to only the previous years' GHG emissions?

In [34]:
pure_GHG=['2015 Total GHG Emissions (Metric Tons CO2e)','2014 Total GHG Emissions (Metric Tons CO2e)',
          '2013 Total GHG Emissions (Metric Tons CO2e)','2012 Total GHG Emissions (Metric Tons CO2e)',
          '2011 Total GHG Emissions (Metric Tons CO2e)']

In [35]:
MSEs=cross_val_score(RF, X[pure_GHG], y[target], cv=10)

print ("Random Forest Regressor MSE on 10-fold CV= %0.2f +/- %0.2f" %(np.mean(MSEs),np.std(MSEs)))

Random Forest Regressor MSE on 10-fold CV= 0.77 +/- 0.26


In [36]:
RMSEs=np.sqrt(cross_val_score(RF, X[pure_GHG], y[target], cv=10))
print ("Random Forest Regressor RMSE on 10-fold CV= %0.2f +/- %0.2f in Metric Tons of CO2" %(np.mean(RMSEs),np.std(RMSEs)))

Random Forest Regressor RMSE on 10-fold CV= 0.87 +/- 0.17 in Metric Tons of CO2


How might this be useful to city planners? Well, perhaps we could turn this into more of a forecasting problem! Let's take the previous year's data, regardless of year, and try and predict the following year.

In [57]:
from sklearn.metrics import mean_squared_error

all_cols=df.columns#Grab all possible columns
years=[x for x in range(2011,2017,1)]
bad_str="Property" #Sometimes these variables sneak into the columns
for k,v in enumerate(years[:-1]):
    print ("Year: ",v)
    #Grab all predictive columns in that year
    predictive_cols=[x for x in all_cols if str(v) in x if bad_str not in x]#Grab only the important columns
    predictive_cols.append("Floor Area")
    
    #Set the target column to that year
    target_col=str(years[k+1])+" Total GHG Emissions (Metric Tons CO2e)"
    X,y=df[predictive_cols],df[target_col]
    #print( X[predictive_cols])
    #print( y[target_col])
    
    if k>0:
        next_predict = (RF.predict(X[predictive_cols]) )
        next_year_RMSE=np.sqrt(mean_squared_error(y,next_predict))
        print ("The Root Mean Squared Error of the previous year's prediction: %0.2f" %(next_year_RMSE))
        
    
    #Run the regression
    R2s=(cross_val_score(RF, X[predictive_cols], y, cv=10))

    R2s=pd.DataFrame({"$R^2$":R2s})
    print ("Random Forest Regressor prediction $R^2$ coefficient on 10-fold CV= %0.2f \
    +/- %0.2f in Metric Tons of CO2 for %i" %(np.mean(R2s),np.std(R2s),years[k+1]))
    
    RF.fit(X[predictive_cols], y)#Fit for the next year.   
    

Year:  2011
Random Forest Regressor prediction $R^2$ coefficient on 10-fold CV= -735.37     +/- 1535.38 in Metric Tons of CO2 for 2012
Year:  2012
The Root Mean Squared Error of the previous year's prediction: 87415.30
Random Forest Regressor prediction $R^2$ coefficient on 10-fold CV= -5.57     +/- 17.52 in Metric Tons of CO2 for 2013
Year:  2013
The Root Mean Squared Error of the previous year's prediction: 2800.06
Random Forest Regressor prediction $R^2$ coefficient on 10-fold CV= -0.22     +/- 1.59 in Metric Tons of CO2 for 2014
Year:  2014
The Root Mean Squared Error of the previous year's prediction: 518.59
Random Forest Regressor prediction $R^2$ coefficient on 10-fold CV= 0.68     +/- 0.25 in Metric Tons of CO2 for 2015
Year:  2015
The Root Mean Squared Error of the previous year's prediction: 367.25
Random Forest Regressor prediction $R^2$ coefficient on 10-fold CV= 0.78     +/- 0.16 in Metric Tons of CO2 for 2016


In [63]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10)

all_cols=df.columns#Grab all possible columns
total_GHGs=[]

RF=RandomForestRegressor(n_estimators=30, criterion='mse',n_jobs=-1)
for k,v in enumerate(years[:-1]):
    RMSEs=[]
    print ("Year: ",v)
    #Grab all predictive columns in that year
    predictive_cols=[x for x in all_cols if str(v) in x if bad_str not in x]#Grab only the important columns
    predictive_cols.append("Floor Area")
    
    #Set the target column to that year
    target_col=str(years[k+1])+" Total GHG Emissions (Metric Tons CO2e)"
    X,y=df[predictive_cols].as_matrix(),df[target_col].as_matrix()
    
    if k>0:
        next_predict = (RF.predict(X) )
        next_year_RMSE=np.sqrt(mean_squared_error(y,next_predict))
        print ("The Root Mean Squared Error of the previous year's prediction: %0.2f" %(next_year_RMSE))
        
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        RF.fit(X_train,y_train)
        predictions=RF.predict(X_test)
        RMSEs.append(np.sqrt(mean_squared_error(y_test,predictions)) )
    print ("Random Forest Regressor prediction RMSE coefficient on 10-fold CV= %0.2f \
    +/- %0.2f in Metric Tons of CO2 for %i" %(np.mean(RMSEs),np.std(RMSEs),years[k+1]))
    

Year:  2011
Random Forest Regressor prediction RMSE coefficient on 10-fold CV= 72097.16     +/- 125820.40 in Metric Tons of CO2 for 2012
Year:  2012
The Root Mean Squared Error of the previous year's prediction: 58032.52
Random Forest Regressor prediction RMSE coefficient on 10-fold CV= 2570.24     +/- 3656.98 in Metric Tons of CO2 for 2013
Year:  2013
The Root Mean Squared Error of the previous year's prediction: 3091.80
Random Forest Regressor prediction RMSE coefficient on 10-fold CV= 794.23     +/- 480.79 in Metric Tons of CO2 for 2014
Year:  2014
The Root Mean Squared Error of the previous year's prediction: 447.07
Random Forest Regressor prediction RMSE coefficient on 10-fold CV= 448.10     +/- 243.80 in Metric Tons of CO2 for 2015
Year:  2015
The Root Mean Squared Error of the previous year's prediction: 364.98
Random Forest Regressor prediction RMSE coefficient on 10-fold CV= 386.60     +/- 251.65 in Metric Tons of CO2 for 2016


It is fairly clear that initially, our model is not that great at forecasting the next year's data (2011 model predicting 2012), but that the model gets fairly more robust over time. This is easy to understand if one looks at the total energy use from 2011 onwards, which includes a precipitous drop, followed by a flat trend.

Onwards, our model becomes fairly robust and does not differ from year to year. This suggests we can make fairly static forecasts with decent accuracy.

Now we can make predictions for 2017! It is a fairly flat model, so a naive approach would probably be just as effective, but we have gone this far already with the Random Forest Regressor...

In [60]:
predictive_cols=[x for x in all_cols if str(2016) in x if bad_str not in x]
predictive_cols.append("Floor Area")
#Set the target column to that year
target_col=str(2016)+" Total GHG Emissions (Metric Tons CO2e)"
X=df[predictive_cols]
X["Predicted 2017 Total GHG Emissions (Metric Tons CO2e)"]=RF.predict(X)
X.head()

C:\Users\j\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,2016 ENERGY STAR Score,2016 Site EUI (kBtu/ft2),2016 Source EUI (kBtu/ft2),2016 Percent Better than National Median Site EUI,2016 Percentage Better than National Median Source EUI,2016 Total GHG Emissions (Metric Tons CO2e),2016 Total GHG Emissions Intensity (kgCO2e/ft2),2016 Weather Normalized Site EUI (kBtu/ft2),2016 Weather Normalized Source EUI (kBtu/ft2),Floor Area,Predicted 2017 Total GHG Emissions (Metric Tons CO2e)
0,83.0,48.5,148.8,20.9,20.9,44.6,4.2,49.0,149.3,10728.0,55.516667
1,83.0,48.7,119.9,-33.2,-33.2,147.4,3.7,51.6,116.4,28300.0,141.670000
2,83.0,27.9,83.2,-43.8,-43.8,24.6,2.3,27.9,83.2,10467.0,24.610000
3,32.0,265.1,631.6,11.7,11.7,739.8,19.8,267.5,634.1,97422.0,641.060000
4,83.0,236.3,467.6,215.8,215.8,510.5,16.1,236.3,467.6,31740.0,503.366667
